In [1]:
import numpy as np
import math
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
import tensorflow as tf
from keras import backend as K
import random

Using TensorFlow backend.


In [2]:
sess = tf.Session()
K.set_session(sess)
K.clear_session()

In [3]:
gt = np.load("outputs/predder.npy")

In [4]:
gt1 = gt[1]['predictions_1']
gt2 = gt[1]['predictions_2']
gt1_proj = gt[1]['predictions_1_proj_tot']
gt2_proj = gt[1]['predictions_2_proj_tot']

In [5]:
pred_1 = np.load("outputs/predictions_1_14.npy")
pred_1_proj = np.load("outputs/predictions_1_proj_14.npy")
pred_2 = np.load("outputs/predictions_2_14.npy")
pred_2_proj = np.load("outputs/predictions_2_proj_14.npy")

labels:  (5, 4)
y_encoded:  (17292, 4)
similarities:  (5, 17292)
bipartite_matches:  (5,)
similarities after bipartite:  (5, 17292)
matches:  (53,)  -  (53,)
similarities after matches:  (5, 17292)

In [6]:
y_true_1 = gt1_proj[:,:,:18]
y_pred_1 = pred_1_proj[:,:,18:]
y_true_2 = gt1_proj[:,:,18:]
y_pred_2 = pred_1_proj[:,:,18:]

In [7]:
batch_size = tf.shape(y_pred_1)[0]

In [8]:
y_true_1d = tf.convert_to_tensor(np.concatenate([y_true_1,y_true_1]),dtype=tf.float32)
y_pred_1d = tf.convert_to_tensor(np.concatenate([y_pred_1,y_pred_1]),dtype=tf.float32)
y_true_2d = tf.convert_to_tensor(np.concatenate([y_true_2,y_true_2]),dtype=tf.float32)
y_pred_2d = tf.convert_to_tensor(np.concatenate([y_pred_2,y_pred_2]),dtype=tf.float32)

y_pred = y_pred_1d
y_true = y_true_1d

In [9]:
neg_pos_ratio = 3
n_neg_min = 0
alpha = 1.0

def smooth_L1_loss(y_true, y_pred):
    '''
    Compute smooth L1 loss, see references.

    Arguments:
        y_true (nD tensor): A TensorFlow tensor of any shape containing the ground truth data.
            In this context, the expected tensor has shape `(batch_size, #boxes, 4)` and
            contains the ground truth bounding box coordinates, where the last dimension
            contains `(xmin, xmax, ymin, ymax)`.
        y_pred (nD tensor): A TensorFlow tensor of identical structure to `y_true` containing
            the predicted data, in this context the predicted bounding box coordinates.

    Returns:
        The smooth L1 loss, a nD-1 Tensorflow tensor. In this context a 2D tensor
        of shape (batch, n_boxes_total).

    References:
        https://arxiv.org/abs/1504.08083
    '''
    absolute_loss = tf.abs(y_true - y_pred)
    square_loss = 0.5 * (y_true - y_pred)**2
    l1_loss = tf.where(tf.less(absolute_loss, 1.0), square_loss, absolute_loss - 0.5)
    return tf.reduce_sum(l1_loss, axis=-1)

def log_loss(y_true, y_pred):
    '''
    Compute the softmax log loss.

    Arguments:
        y_true (nD tensor): A TensorFlow tensor of any shape containing the ground truth data.
            In this context, the expected tensor has shape (batch_size, #boxes, #classes)
            and contains the ground truth bounding box categories.
        y_pred (nD tensor): A TensorFlow tensor of identical structure to `y_true` containing
            the predicted data, in this context the predicted bounding box categories.

    Returns:
        The softmax log loss, a nD-1 Tensorflow tensor. In this context a 2D tensor
        of shape (batch, n_boxes_total).
    '''
    # Make sure that `y_pred` doesn't contain any zeros (which would break the log function)
    y_pred = tf.maximum(y_pred, 1e-15)
    # Compute the log loss
    log_loss = -tf.reduce_sum(y_true * tf.log(y_pred), axis=-1)
    return log_loss


In [10]:
batch_size = tf.shape(y_pred)[0] # Output dtype: tf.int32
n_boxes = tf.shape(y_pred)[1] # Output dtype: tf.int32, note that `n_boxes` in this context denotes the total number of boxes per image, not the number of boxes per cell.

# 1: Compute the losses for class and box predictions for every box.

classification_loss = tf.to_float(log_loss(y_true[:,:,:-16], y_pred[:,:,:-16])) # Output shape: (batch_size, n_boxes)
localization_loss = tf.to_float(smooth_L1_loss(y_true[:,:,-16:-12], y_pred[:,:,-16:-12])) # Output shape: (batch_size, n_boxes)

print("classification_loss: ",K.eval(classification_loss).shape)
print("localization_loss: ",K.eval(localization_loss).shape)

negatives = y_true[:,:,0] # Tensor of shape (batch_size, n_boxes)
positives = tf.to_float(tf.reduce_max(y_true[:,:,1:-16], axis=-1)) # Tensor of shape (batch_size, n_boxes)


classification_loss:  (2, 17292)
localization_loss:  (2, 17292)


In [11]:
n_positive = tf.reduce_sum(positives)
pos_class_loss = tf.reduce_sum(classification_loss * positives, axis=-1) # Tensor of shape (batch_size,)
neg_class_loss_all = classification_loss * negatives # Tensor of shape (batch_size, n_boxes)
n_neg_losses = tf.count_nonzero(neg_class_loss_all, dtype=tf.int32) # The number of non-zero loss entries in `neg_class_loss_all`
print("n_positive: ",K.eval(n_positive).shape)
print("pos_class_loss: ",K.eval(pos_class_loss).shape)
print("neg_class_loss_all: ",K.eval(neg_class_loss_all).shape)
print("n_neg_losses: ",K.eval(n_neg_losses).shape)


n_positive:  ()
pos_class_loss:  (2,)
neg_class_loss_all:  (2, 17292)
n_neg_losses:  ()


In [12]:
n_negative_keep = tf.minimum(tf.maximum(neg_pos_ratio * tf.to_int32(n_positive), n_neg_min), n_neg_losses)
print("n_negative_keep: ",K.eval(n_negative_keep).shape)

# In the unlikely case when either (1) there are no negative ground truth boxes at all
# or (2) the classification loss for all negative boxes is zero, return zero as the `neg_class_loss`.
def f1():
    return tf.zeros([batch_size])
# Otherwise compute the negative loss.
def f2():
    # Now we'll identify the top-k (where k == `n_negative_keep`) boxes with the highest confidence loss that
    # belong to the background class in the ground truth data. Note that this doesn't necessarily mean that the model
    # predicted the wrong class for those boxes, it just means that the loss for those boxes is the highest.

    # To do this, we reshape `neg_class_loss_all` to 1D...
    neg_class_loss_all_1D = tf.reshape(neg_class_loss_all, [-1]) # Tensor of shape (batch_size * n_boxes,)
    # ...and then we get the indices for the `n_negative_keep` boxes with the highest loss out of those...
    values, indices = tf.nn.top_k(neg_class_loss_all_1D,
                                  k=n_negative_keep,
                                  sorted=False) # We don't need them sorted.
    # ...and with these indices we'll create a mask...
    negatives_keep = tf.scatter_nd(indices=tf.expand_dims(indices, axis=1),
                                   updates=tf.ones_like(indices, dtype=tf.int32),
                                   shape=tf.shape(neg_class_loss_all_1D)) # Tensor of shape (batch_size * n_boxes,)
    negatives_keep = tf.to_float(tf.reshape(negatives_keep, [batch_size, n_boxes])) # Tensor of shape (batch_size, n_boxes)
    # ...and use it to keep only those boxes and mask all other classification losses
    neg_class_loss = tf.reduce_sum(classification_loss * negatives_keep, axis=-1) # Tensor of shape (batch_size,)
    return neg_class_loss

neg_class_loss = tf.cond(tf.equal(n_neg_losses, tf.constant(0)), f1, f2)
print("neg_class_loss: ",K.eval(neg_class_loss).shape)

class_loss = pos_class_loss + neg_class_loss # Tensor of shape (batch_size,)
print("class_loss: ",K.eval(neg_class_loss).shape)

# 3: Compute the localization loss for the positive targets.
#    We don't compute a localization loss for negative predicted boxes (obviously: there are no ground truth boxes they would correspond to).

loc_loss = tf.reduce_sum(localization_loss * positives, axis=-1) # Tensor of shape (batch_size,)
print("loc_loss: ",K.eval(loc_loss).shape)

# 4: Compute the total loss.

total_loss = (class_loss + alpha * loc_loss) / tf.maximum(1.0, n_positive) # In case `n_positive == 0`
# Keras has the annoying habit of dividing the loss by the batch size, which sucks in our case
# because the relevant criterion to average our loss over is the number of positive boxes in the batch
# (by which we're dividing in the line above), not the batch size. So in order to revert Keras' averaging
# over the batch size, we'll have to multiply by it.
print("total_loss: ",K.eval(total_loss).shape)

total_loss = total_loss * tf.to_float(batch_size)
print("total_loss: ",K.eval(total_loss).shape)

n_negative_keep:  ()
neg_class_loss:  (2,)
class_loss:  (2,)
loc_loss:  (2,)
total_loss:  (2,)
total_loss:  (2,)


In [13]:
# total_loss.set_shape((None,))


In [14]:
K.eval(total_loss)


array([12.947088, 11.466916], dtype=float32)